# Data Augmentation

In [28]:
import pandas as pd
import ast
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from collections import defaultdict

load_dotenv()

True

Load the ATECO 2025 leaf-level index.

In [ ]:
df = pd.read_csv("data/ateco_2025_leaf.csv")

---

## GPT Augmentation

In [13]:
client = OpenAI()
model_id: str = "gpt-4.1-mini"

system = """Sei un generatore di descrizioni di attività svolte da aziende e professionisti.

Ecco cosa devi fare:
* Ricevi in input una classificazione economica (titolo + dettagli).
* Generi 10 esempi di brevissime descrizioni di attività svolte da aziende, coerenti con la classificazione.
* Non generi testo che non sia una delle descrizioni.

Informazioni importanti:
* Focalizzati sull'attività indicata, non generare descrizioni con attività diverse (pui usare sinonimi).

Alcuni esempi di descrizioni:
i) Commercio dettaglio mobili usati oggetti antiquariato;
ii) Restauro conservazione;
iii) Tende sole esterno;
iv) Consulenza sistemi Gestione;
v) Pompaggio calcestruzzo;
vi) Gommista.

Formatta l'output come una lista Python."""

Test the model.

In [25]:
response = client.responses.create(
    model=model_id,
    input=[
        {
            "role": "developer",
            "content": system
        },
        {
            "role": "user",
            "content": df["prompts"][1194]
        }
    ]
)

out_list = ast.literal_eval(response.output_text)

Run the augmentation script.

In [ ]:
out_dict = {}

output_path = "data/augmented.json"

if os.path.exists(output_path):
    with open(output_path, "r") as f:
        try:
            out_dict = json.load(f)
        except json.JSONDecodeError:
            out_dict = {}

for i, row in df.iterrows():
    code = row["code"]
    prompt = row["prompts"]

    response = client.responses.create(
        model=model_id,
        input=[
            {
                "role": "developer",
                "content": system
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    try:
        out_list = ast.literal_eval(response.output_text)

    except:
        out_list = str(response.output_text)
    
    out_dict[code] = out_list

    with open(output_path, "w") as f:
        json.dump(out_dict, f, indent=2)
